<a href="https://colab.research.google.com/github/anonyms/FantasyFootball/blob/main/LeagueData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from pandas import json_normalize
import pandas as pd
import numpy as np
import json
import requests
import datetime
from google.colab import data_table
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
import itertools as it

In [2]:
FPL_URL = "https://fantasy.premierleague.com/api/"
LOGIN_URL = "https://users.premierleague.com/accounts/login/"
USER_SUMMARY_SUBURL = "element-summary/"
LEAGUE_CLASSIC_STANDING_SUBURL = "leagues-classic/"
TEAM_ENTRY_SUBURL = "entry/"
GAMEWEEK_SUBURL = "event/"
PLAYERS_INFO_SUBURL = "bootstrap-static/"
USERNAME = XXX
PASSWORD =  XXXX
USER_SUMMARY_URL = FPL_URL + USER_SUMMARY_SUBURL
PLAYERS_INFO_URL = FPL_URL + PLAYERS_INFO_SUBURL
session = requests.session()

In [3]:
url = LOGIN_URL
payload = {
 'password': PASSWORD,
 'login': USERNAME,
 'redirect_uri': 'https://fantasy.premierleague.com/a/login',
 'app': 'plfpl-web'
}
session.post(url, data=payload)

<Response [200]>

In [4]:
# Get basic Standings View for a league
def getStandings(league_id):
  ## Create URL for current standings - given league id
  league_id = str(league_id)
  call_url = FPL_URL + LEAGUE_CLASSIC_STANDING_SUBURL + league_id + "/standings/"
  response = session.get(call_url)
  jsonResponse = response.json()
  ## Get standings
  standings = json_normalize(jsonResponse['standings'], 'results')
  return standings

# Get Id for all league participants (managers) - through the standings view
def getManagerIds(league_id):
  ## Create URL for current standings - given league id & where also the manager Ids are present
  league_id_str = str(league_id)
  call_url = FPL_URL + LEAGUE_CLASSIC_STANDING_SUBURL + league_id_str + "/standings/"
  response = session.get(call_url)
  jsonResponse = response.json()
  standings = json_normalize(jsonResponse['standings'], 'results')
  ## Return just Manager ids (entry ids)
  return standings['entry']

## Get Game Week History For a Manager & Team
def getManagerTeamInfoPerWeek(entry_id):
  #Create URL for Manager (entry) histroy
  entry_id = str(entry_id)
  call_url = FPL_URL + TEAM_ENTRY_SUBURL + entry_id + "/history/"
  response = session.get(call_url)
  jsonResponse = response.json()
  gameweek_history = json_normalize(jsonResponse['current'])
  ##Get relevant data
  gameweek_history_picked = gameweek_history.loc[:,["event","points","total_points","value","event_transfers","points_on_bench"]]
  gameweek_history_picked["entry_id"] = int(entry_id)
  ##Return Data frame with whole history
  return gameweek_history_picked

def getAllManagerTeamInfoPerWeek(league_id):
  entry_ids = getManagerIds(league_id)
  result_df = pd.DataFrame()
  for e in entry_ids:
    result_df = result_df.append(getManagerTeamInfoPerWeek(e),ignore_index=True)
  return result_df

def addRank(df, name, groupbys = ["event"], by = "total_points", ascending=False):
  df["league_rank"] = df.groupby(groupbys)[by].rank(ascending=ascending)
  return df

def getManagerNameAndTeamName(league_id):
  league_id = str(league_id)
  call_url = FPL_URL + LEAGUE_CLASSIC_STANDING_SUBURL + league_id + "/standings/"
  response = session.get(call_url)
  jsonResponse = response.json()
  standings = json_normalize(jsonResponse['standings'], 'results')
  player_names = standings.loc[:,['entry','player_name','entry_name']]
  return player_names

## Get data for a leauge on weekly basis (based on Gameweek history)
def getWeeklyLeagueData(league_id):
  manager_names = getManagerNameAndTeamName(league_id)
  df = getAllManagerTeamInfoPerWeek(league_id)
  df = addRank(df,"league_rank")
  df = pd.merge(df,manager_names,how='inner',left_on='entry_id',right_on='entry')
  df['avg_week_points'] = df.groupby('event')['points'].transform('mean')
  df['diff_from_avg_points'] = df['points'] - df['avg_week_points']
  ## Get current standings as separate column - in order to filter on it later
  current_standings = getStandings(league_id)
  current_standings = current_standings.loc[:,["rank","entry"]]
  df_with_current_standings = pd.merge(df,current_standings,how="inner",on="entry")
  df_with_current_standings = df_with_current_standings.rename(columns={"rank": "current_rank","event": "gameweek"})
  return df_with_current_standings

def getAllPLTeamsData():
  call_url = PLAYERS_INFO_URL
  response = session.get(call_url)
  jsonResponse = response.json()
  pl_teams = json_normalize(jsonResponse['teams'])
  return pl_teams

def getBasicPlayerData():
  call_url = PLAYERS_INFO_URL
  response = session.get(call_url)
  jsonResponse = response.json()
  players = json_normalize(jsonResponse['elements'])
  players = players.loc[:,["code","id","first_name","second_name","web_name","total_points","minutes","now_cost","value_season","team","element_type"]]
  players["value_season"] = players["value_season"].astype('float')
  pl_teams = getAllPLTeamsData()
  pl_teams = pl_teams.loc[:,["id","name"]]
  pl_teams = pl_teams.rename(columns={"id": "team_id", "name": "team_name"})
  players = pd.merge(players,pl_teams,how='left',left_on='team',right_on='team_id')
  players = players.drop(['team'], axis=1)
  return players

def getAllPlayerData():
  call_url = PLAYERS_INFO_URL
  response = session.get(call_url)
  jsonResponse = response.json()
  players = json_normalize(jsonResponse['elements'])
  players["value_season"] = players["value_season"].astype('float')
  return players

## Get all picks (current team) for a given manager (id) for a given gameweek
## ATTENTION: Return empty data frame if no values found for given game week
def getManagerPicks(manager_id, gameweek_nb):
  manager_id_str = str(manager_id)
  call_url = FPL_URL + TEAM_ENTRY_SUBURL + manager_id_str + "/event/" + str(gameweek_nb) + "/picks/"
  response = session.get(call_url)
  jsonResponse = response.json()
  if "picks" in list(jsonResponse.keys()):
    picks = json_normalize(jsonResponse['picks'])
    picks = picks.loc[:,["element","is_captain"]]
    picks["manager_id"] = manager_id
    picks["gameweek"] = gameweek_nb
  else:
    picks = pd.DataFrame()
  return picks

## Get the NUmber of the last Gameweek that gave points
def getLastGameWeek(league_id):
  manager_ids = getManagerIds(league_id)  ## Need manager ids to get GameWeek History
  random_manager_id = manager_ids.sample().values[0] ##Pick a random one just to extract one
  gameweek_history = getManagerTeamInfoPerWeek(random_manager_id) #Get gameweek history for said manager
  return gameweek_history[gameweek_history['points'] >0].max()['event'] ##Return Max game week with points ie Last valid gameweek

## Get All Managers Current Players for a league
def getAllManagersCurrentPicks(league_id):
  gameweek_nb = getLastGameWeek(league_id) ## Get last gameweek nb, to get current squands
  entry_ids = getManagerIds(league_id)
  result_df = pd.DataFrame()
  for e in entry_ids:
    result_df = result_df.append(getManagerPicks(e,gameweek_nb),ignore_index=True) ## Iterate over all managers to obtain team per manager
  return result_df

## Get All Managers Picked Players per week with points
## ATTENTION: Return empty data frame if no values found for given game week - defined in getManagerPicks
def getAllManagersPicksPerGameWeek(league_id):
  gameweek_nb = getLastGameWeek(league_id) ## Get last gameweek nb, to know how far to iterate
  entry_ids = getManagerIds(league_id)
  result_df = pd.DataFrame()
  for i in range(1,gameweek_nb+1):
    print("Checking for Gameweek: "+str(i))
    for e in entry_ids:
      result_df = result_df.append(getManagerPicks(e,i),ignore_index=True) ## Iterate over all managers to obtain team per manager
  return result_df

## Any result data frame with a PL Player id, can be joined to get basic stats (name, web name etc)
def joinBasicPLPlayerStats(df, df_player_id = "element"):
  players = getBasicPlayerData()
  new_df = pd.merge(df,players,how='left',left_on=df_player_id,right_on='id')
  return new_df

## Any result data frame with a manager id, can be joined to get manager name and manager team name
def joinManagerDetails(df, league_id, df_manager_id = "manager_id"):
  standings = getStandings(league_id)
  new_df = pd.merge(df,standings,how='left',left_on=df_manager_id,right_on='entry')
  return new_df

##Get Current PL Players per Manager and Manager Team for last completed Gameweek
def currentLeaguePlayersWBasicStats(league_id):
  allpicks = getAllManagersCurrentPicks(league_id)
  allpicks_wplayers = joinBasicPLPlayerStats(allpicks)
  allpicks_wplayers_wmanagers = joinManagerDetails(allpicks_wplayers,league_id)
  allpicks_wplayers_wmanagers = allpicks_wplayers_wmanagers.loc[:,["element","is_captain","web_name","team_name","player_name","entry_name"]]
  return allpicks_wplayers_wmanagers

## Get Element Type id and Name (definitation)
def getElementTypeDefinitions():
  call_url = PLAYERS_INFO_URL
  response = session.get(call_url)
  jsonResponse = response.json()
  element_types = json_normalize(jsonResponse['element_types'])
  element_types = element_types.loc[:,["id","singular_name"]]
  element_types = element_types.rename(columns={"id": "element_type_id","singular_name": "element_type_name"})
  return element_types

## Get stats for PL Player - points & in dream team - for given gameweek
def getPLPlayersPointsForWeek(gameweek_nb):
  ## Use the /event/gameweek/live/ url for detail
  call_url =  FPL_URL+ GAMEWEEK_SUBURL + str(gameweek_nb) + "/live/"
  response = session.get(call_url)
  jsonResponse = response.json()
  players = json_normalize(jsonResponse['elements'])
  players = players.loc[:,["id","stats.total_points","stats.in_dreamteam"]]
  players = players.rename(columns={"id": "plplayer_id","stats.total_points": "gameweek_points", "stats.in_dreamteam": "in_dreamteam"})
  players["gameweek"] = gameweek_nb
  return players

## Get stats for PL Player - points & in dream team - per gameweek
## Need to passs number for gameweeks (use getLastGameweeks)
def getPLPlayersPointsPerWeek(nb_gameweeks):
  result_df = pd.DataFrame()
  for i in range(1,nb_gameweeks+1):
    result_df = result_df.append(getPLPlayersPointsForWeek(i),ignore_index=True) ## Iterate over all gameweeks and get player stats
  return result_df

##Get Weekly PL Players per Manager and Manager Team until last completed Gameweek
def weeklyLeaguePlayersWBasicStats(league_id):
  allpicks = getAllManagersPicksPerGameWeek(league_id)
  allpicks_wplayers = joinBasicPLPlayerStats(allpicks)
  allpicks_wplayers_wmanagers = joinManagerDetails(allpicks_wplayers,league_id)
  element_type = getElementTypeDefinitions()
  allpicks_wplayers_wmanagers = pd.merge(allpicks_wplayers_wmanagers,element_type,how="inner",left_on = "element_type",right_on="element_type_id")
  nb_gameweeks = getLastGameWeek(league_id)
  weekly_plplayer_stats = getPLPlayersPointsPerWeek(nb_gameweeks)
  allpicks_wplayers_wmanagers_wplplayers = pd.merge(allpicks_wplayers_wmanagers,weekly_plplayer_stats,how="inner",left_on=["element","gameweek"],right_on=["plplayer_id","gameweek"])
  allpicks_wplayers_wmanagers_wplplayers = allpicks_wplayers_wmanagers_wplplayers.loc[:,["element","is_captain","web_name","team_name","player_name","entry_name","gameweek","element_type_name","gameweek_points","in_dreamteam"]]
  return allpicks_wplayers_wmanagers_wplplayers

In [5]:
allpicks_wplayers_wmanagers = weeklyLeaguePlayersWBasicStats(672214)
league_weekly_data = getWeeklyLeagueData(672214)

Checking for Gameweek: 1
Checking for Gameweek: 2
Checking for Gameweek: 3
Checking for Gameweek: 4
Checking for Gameweek: 5
Checking for Gameweek: 6
Checking for Gameweek: 7
Checking for Gameweek: 8
Checking for Gameweek: 9
Checking for Gameweek: 10
Checking for Gameweek: 11
Checking for Gameweek: 12
Checking for Gameweek: 13
Checking for Gameweek: 14
Checking for Gameweek: 15
Checking for Gameweek: 16
Checking for Gameweek: 17
Checking for Gameweek: 18
Checking for Gameweek: 19
Checking for Gameweek: 20
Checking for Gameweek: 21


In [6]:
last_gameweek = getLastGameWeek(672214)
allpicks_wplayers_wmanagers["last_gameweek"] = last_gameweek
league_weekly_data["last_gameweek"] = last_gameweek
league_weekly_data["lastgameweek_status"] = league_weekly_data["last_gameweek"] == league_weekly_data["gameweek"]
allpicks_wplayers_wmanagers["lastgameweek_status"] = allpicks_wplayers_wmanagers["last_gameweek"] == allpicks_wplayers_wmanagers["gameweek"]

In [7]:
data_table.DataTable(allpicks_wplayers_wmanagers, include_index=False, num_rows_per_page=10,max_columns =55)

,element,is_captain,web_name,team_name,player_name,entry_name,gameweek,element_type_name,gameweek_points,in_dreamteam,last_gameweek,lastgameweek_status
0,363,False,McCarthy,Southampton,Michael Pennick,Pick Nope,1,Goalkeeper,2,False,21,False
1,363,False,McCarthy,Southampton,Torjus Mannes,Ziyech of glory,1,Goalkeeper,2,False,21,False
2,363,False,McCarthy,Southampton,Michel Nienhuis,King in the North,1,Goalkeeper,2,False,21,False
3,363,False,McCarthy,Southampton,Halldor Skard Jr,Halldorpool,1,Goalkeeper,2,False,21,False
4,363,False,McCarthy,Southampton,Øystein Strand Teslo,FC Oysters,1,Goalkeeper,2,False,21,False
...,...,...,...,...,...,...,...,...,...,...,...,...
4720,166,False,Richarlison,Everton,Kjetil Kjerstad,Nine Lives,21,Forward,2,False,21,True
4721,166,True,Richarlison,Everton,Stephan Sinnige,AC San Marco,21,Forward,2,False,21,True
4722,166,False,Richarlison,Everton,Torbjørn Bjering,Team Beam,21,Forward,2,False,21,True
4723,118,False,Abraham,Chelsea,Kjetil Kjerstad,Nine Lives,21,Forward,1,False,21,True


In [8]:
##list(jsonResponse.keys())

In [9]:
def colnum_string(n):
    string = ""
    while n > 0:
        n, remainder = divmod(n - 1, 26)
        string = chr(65 + remainder) + string
    return string

def getRange(df):
  range_string = "A1:"
  nb_row = len(df)+1
  nb_row_str = str(nb_row)
  nb_columns = len(df.columns)
  nb_columns_str = colnum_string(nb_columns)
  range_string = range_string + nb_columns_str + nb_row_str
  return range_string

def myconverter(obj):
  if isinstance(obj, np.integer):
      return int(obj)
  elif isinstance(obj, np.floating):
      return float(obj)
  elif isinstance(obj, np.bool_):
      return int(obj)
  elif isinstance(obj, datetime.datetime):
        return obj.__str__()
  elif isinstance(obj, datetime.date):
        return obj.__str__()
  else:
    return obj

def updateWorkSheetWithDF(wb,df,sheetname):
  worksheet = wb.worksheet(sheetname)
  range = getRange(df)
  cell_list = worksheet.range(range)
  row_nb = -1
  col_nb = 0
  for cell in cell_list:
    if col_nb == len(df.columns):
      col_nb = 0
      row_nb = row_nb + 1
    if row_nb == -1:
      cell.value = df.columns[col_nb]
    else:
      try:
        tmp = df.iloc[row_nb,col_nb]
      except Exception as e:
        print(row_nb)
        print(col_nb)
      cell.value = myconverter(tmp)
    col_nb = col_nb + 1
  worksheet.update_cells(cell_list)

In [10]:
allpicks_wplayers_wmanagers

,element,is_captain,web_name,team_name,player_name,entry_name,gameweek,element_type_name,gameweek_points,in_dreamteam,last_gameweek,lastgameweek_status
0,363,False,McCarthy,Southampton,Michael Pennick,Pick Nope,1,Goalkeeper,2,False,21,False
1,363,False,McCarthy,Southampton,Torjus Mannes,Ziyech of glory,1,Goalkeeper,2,False,21,False
2,363,False,McCarthy,Southampton,Michel Nienhuis,King in the North,1,Goalkeeper,2,False,21,False
3,363,False,McCarthy,Southampton,Halldor Skard Jr,Halldorpool,1,Goalkeeper,2,False,21,False
4,363,False,McCarthy,Southampton,Øystein Strand Teslo,FC Oysters,1,Goalkeeper,2,False,21,False
...,...,...,...,...,...,...,...,...,...,...,...,...
4720,166,False,Richarlison,Everton,Kjetil Kjerstad,Nine Lives,21,Forward,2,False,21,True
4721,166,True,Richarlison,Everton,Stephan Sinnige,AC San Marco,21,Forward,2,False,21,True
4722,166,False,Richarlison,Everton,Torbjørn Bjering,Team Beam,21,Forward,2,False,21,True
4723,118,False,Abraham,Chelsea,Kjetil Kjerstad,Nine Lives,21,Forward,1,False,21,True


In [11]:
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

## Set worksheet to get data from
WorkbookName = "KPIRangers"
SheetName1 = "WeeklyDataPerManager"
SheetName2 = "WeeklyTeamPerManager"

wb = gc.open(WorkbookName)
sheet1 = wb.worksheet(SheetName1)
sheet2 = wb.worksheet(SheetName2)
updateWorkSheetWithDF(wb,league_weekly_data,SheetName1)
updateWorkSheetWithDF(wb,allpicks_wplayers_wmanagers,SheetName2)
##sheet2.update(allpicks_wplayers_wmanagers)